In [2]:
import sqlite3
import pandas as pd
from tkinter import Tk, Label, Button, Entry, StringVar, messagebox, Toplevel, Scrollbar, HORIZONTAL, VERTICAL
from tkinter import ttk

# function to check availability and other constraints
def check_constraints(data, conn):
    cursor = conn.cursor()

    # check pet type
    if data["Pet Type"] and data["Pet Type"].capitalize() not in ["Dog", "Cat", "Rabbit"]:
        messagebox.showerror("Error", "Pet Type must be 'Dog', 'Cat', or 'Rabbit'.")
        return False

    return True

# function to load csv data into sqlite database
def load_data_to_db(csv_file, db_file="pet_grooming.db"):
    conn = sqlite3.connect(db_file)
    df = pd.read_csv(csv_file)
    df.to_sql('PetGroomingData', conn, if_exists='replace', index=False)
    conn.close()

# function to view all past records with scrollbars and nil for nan values
def view_records():
    records_window = Toplevel()
    records_window.title("View Past Records")

    data = pd.read_csv("pet_grooming_data_final.csv").fillna("nil")

    tree = ttk.Treeview(records_window)
    tree["columns"] = list(data.columns)
    tree["show"] = "headings"

    for col in data.columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    for _, row in data.iterrows():
        tree.insert("", "end", values=list(row))

    h_scroll = Scrollbar(records_window, orient=HORIZONTAL, command=tree.xview)
    v_scroll = Scrollbar(records_window, orient=VERTICAL, command=tree.yview)
    tree.configure(xscrollcommand=h_scroll.set, yscrollcommand=v_scroll.set)

    h_scroll.pack(side="bottom", fill="x")
    v_scroll.pack(side="right", fill="y")
    tree.pack(fill="both", expand=True)

# function to search records with scrollbars and nil for nan values
def search_records():
    def search():
        search_term = search_entry.get()
        search_data = pd.read_csv("pet_grooming_data_final.csv").fillna("nil")
        result = search_data[search_data['Customer ID'].astype(str).str.contains(search_term, case=False) |
                             search_data['Pet Name'].str.contains(search_term, case=False)]

        for item in tree.get_children():
            tree.delete(item)
        for _, row in result.iterrows():
            tree.insert("", "end", values=list(row))

    search_window = Toplevel()
    search_window.title("Search Past Records")

    Label(search_window, text="Search by Customer ID or Pet Name:").pack()
    search_entry = Entry(search_window)
    search_entry.pack()
    Button(search_window, text="Search", command=search).pack()

    tree = ttk.Treeview(search_window)
    data = pd.read_csv("pet_grooming_data_final.csv").fillna("nil")
    tree["columns"] = list(data.columns)
    tree["show"] = "headings"

    for col in data.columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    h_scroll = Scrollbar(search_window, orient=HORIZONTAL, command=tree.xview)
    v_scroll = Scrollbar(search_window, orient=VERTICAL, command=tree.yview)
    tree.configure(xscrollcommand=h_scroll.set, yscrollcommand=v_scroll.set)

    h_scroll.pack(side="bottom", fill="x")
    v_scroll.pack(side="right", fill="y")
    tree.pack(fill="both", expand=True)

# function to insert a new record and save to csv
def insert_record():
    def submit_data():
        # collect data from the form fields allowing empty fields
        data = {
            "Owner Name": owner_name.get(),
            "Pet Name": pet_name.get(),
            "Pet Type": pet_type.get(),
            "Gender": gender.get(),
            "Owner Address": owner_address.get(),
            "Grooming Service Type": grooming_service.get(),
            "Service Date": service_date.get(),
            "Service Price": service_price.get(),
            "Customer ID": customer_id.get(),
            "Groomer ID": groomer_id.get(),
            "Appointment ID": appointment_id.get(),
            "Pet ID": pet_id.get(),
            "Pet Breed": pet_breed.get(),
            "Preferred Groomer Years of Experience": experience.get(),
            "Groomer Gender": groomer_gender.get(),
            "Appointment Status": appointment_status.get(),
            "Appointment Date": appointment_date.get(),
            "Time": time.get()
        }

        conn = sqlite3.connect("pet_grooming.db")

        if check_constraints(data, conn):
            cursor = conn.cursor()
            cursor.execute('''INSERT INTO PetGroomingData VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', tuple(data.values()))
            conn.commit()

            new_record = pd.DataFrame([data])
            new_record.to_csv("pet_grooming_data_final.csv", mode='a', header=False, index=False)

            messagebox.showinfo("Success", "Thank you! Record submitted successfully!")
            insert_window.destroy()  # close the insert window after submission
        conn.close()

    insert_window = Toplevel()
    insert_window.title("Insert New Record")

    fields = [
        "Owner Name", "Pet Name", "Pet Type", "Gender", "Owner Address",
        "Grooming Service Type", "Service Date", "Service Price", "Customer ID",
        "Groomer ID", "Appointment ID", "Pet ID", "Pet Breed",
        "Preferred Groomer Years of Experience", "Groomer Gender", "Appointment Status",
        "Appointment Date", "Time"
    ]

    variables = [StringVar() for _ in fields]
    for i, field in enumerate(fields):
        Label(insert_window, text=field).grid(row=i, column=0, padx=5, pady=5)
        Entry(insert_window, textvariable=variables[i]).grid(row=i, column=1, padx=5, pady=5)

    owner_name, pet_name, pet_type, gender, owner_address, grooming_service, service_date, \
    service_price, customer_id, groomer_id, appointment_id, pet_id, pet_breed, \
    experience, groomer_gender, appointment_status, appointment_date, time = variables

    Button(insert_window, text="Submit", command=submit_data).grid(row=len(fields), column=1, pady=10)

# main application window
def main():
    root = Tk()
    root.title("Pet Grooming Database System")

    Button(root, text="Insert New Record", command=insert_record).pack(pady=10)
    Button(root, text="View Past Records", command=view_records).pack(pady=10)
    Button(root, text="Search Past Records", command=search_records).pack(pady=10)

    root.mainloop()

load_data_to_db("pet_grooming_data_final.csv")
main()
